In [1]:
import arcpy
import os 

In [4]:
inDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject"
inRasterTN = os.path.join(inDir, "Luwen_Nutrient", "TN_Annual_delTotal_header_kgcellday.tif")
inRasterTP = os.path.join(inDir, "Luwen_Nutrient", "TP_Annual_delTotal_header_kgcellday.tif")
Huronbasin  = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWatershed_LH_NAD1983_NASA_Invasive.shp"

outDir = os.path.join(inDir, "Luwen_Nutrient")

arcpy.env.workspace = outDir
arcpy.env.overwriteOutput = True

def prep_mask_for_raster(raster_path, mask_fc):
    """Return a dissolved+projected polygon mask in memory that matches the raster CRS."""
    arcpy.management.RepairGeometry(mask_fc)

    # dissolve in memory (handles multipart/overlaps)
    mask_diss = r"in_memory\mask_diss"
    if arcpy.Exists(mask_diss):
        arcpy.management.Delete(mask_diss)
    arcpy.management.Dissolve(mask_fc, mask_diss)

    # project dissolved mask to raster SR (if needed)
    ras_sr = arcpy.Describe(raster_path).spatialReference
    mask_proj = r"in_memory\mask_proj"
    if arcpy.Exists(mask_proj):
        arcpy.management.Delete(mask_proj)

    if ras_sr and ras_sr.factoryCode not in (0, None):
        arcpy.management.Project(mask_diss, mask_proj, ras_sr)
        return mask_proj
    else:
        # raster has no SR? just use dissolved mask
        return mask_diss

def clip_raster_to_mask(in_raster, mask_fc, out_path):
    # keep grid aligned
    arcpy.env.snapRaster = in_raster
    arcpy.env.cellSize   = in_raster

    mask_for_this_raster = prep_mask_for_raster(in_raster, mask_fc)

    arcpy.management.Clip(
        in_raster=in_raster,
        rectangle="#",                       # use mask extent
        out_raster=out_path,
        in_template_dataset=mask_for_this_raster,
        nodata_value="NoData",
        clipping_geometry="ClippingGeometry",# <- clip to polygon shape
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )
    print(f"✔ Clipped: {os.path.basename(in_raster)} -> {out_path}")

# make sure outDir exists
os.makedirs(outDir, exist_ok=True)

tn_out = os.path.join(outDir, "TN_Annual_delTotal_header_kgcellday_Huron2.tif")
tp_out = os.path.join(outDir, "TP_Annual_delTotal_header_kgcellday_Huron2.tif")

clip_raster_to_mask(inRasterTN, Huronbasin, tn_out)
clip_raster_to_mask(inRasterTP, Huronbasin, tp_out)

# clean up memory
arcpy.management.Delete("in_memory")

✔ Clipped: TN_Annual_delTotal_header_kgcellday.tif -> D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient\TN_Annual_delTotal_header_kgcellday_Huron2.tif
✔ Clipped: TP_Annual_delTotal_header_kgcellday.tif -> D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient\TP_Annual_delTotal_header_kgcellday_Huron2.tif


<Result 'true'>